# Why do IRL and IRR files have similar, if not the same, insertions?

Could be because the transposon and adapter tags were incorrect from the IAS pipeline, and on top of that they were incorrectly used in cutadapt (not using reverse complement). I have output from bowtie2 that summarizes the mapping results and will need to dynamically parse the output file for mapping qualities

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
bowtie_dir = Path("/project/cs-myers/MathewF/projects/Laura-SB-Analysis/NetCIS/toy-data/2020_SB-bowtie/")

In [5]:
list_df = []
for file in bowtie_dir.iterdir():
    cell_type, sample_id, tumor_tmp = file.name.split("-")
    tumor_type, lib_tmp = tumor_tmp.split("_")
    library_type = lib_tmp.split(".")[0]
    tmp_dict = {"cell_type": cell_type, "sample_id": sample_id, "tumor_type": tumor_type, "tumor_library": library_type}
    with open(file) as f:
        for line in f:
            line = line.strip()
            if line.find("reads; of these:") != -1:
                tmp_dict["total-reads"] = [int(line.split(" ")[0])]
            elif line.find("were paired; of these:") != -1:
                tmp_dict["paired-reads"] = [int(line.split(" ")[0])]
            elif line.find("aligned concordantly 0 times") != -1:
                tmp_dict["concordant-reads-0"] = [int(line.split(" ")[0])]
            elif line.find("aligned concordantly exactly 1 time") != -1:
                tmp_dict["concordant-reads-1"] = [int(line.split(" ")[0])]
            elif line.find("aligned concordantly >1 times") != -1:
                tmp_dict["concordant-reads->1"] = [int(line.split(" ")[0])]
            elif line.find("aligned discordantly 1 time") != -1:
                tmp_dict["discordant-reads-1"] = [int(line.split(" ")[0])]
            else:
                pass
    tmp_df = pd.DataFrame.from_dict(tmp_dict)
    list_df.append(tmp_df)
bowtie_df = pd.concat(list_df, ignore_index=True)
bowtie_df["aligned-reads"] = bowtie_df["concordant-reads-1"] + bowtie_df["concordant-reads->1"] + bowtie_df["discordant-reads-1"]
display(bowtie_df)

,cell_type,sample_id,tumor_type,tumor_library,total-reads,paired-reads,concordant-reads-0,concordant-reads-1,concordant-reads->1,discordant-reads-1,aligned-reads
0,B16,1_1,RT,IRR,1529,1523,1104,394,25,3,422
1,B16,1_1,S,IRR,1810,1795,1625,153,17,1,171
2,B16,1_1,RT,IRL,1733,1733,1706,4,23,0,27
3,B16,1_1,S,IRL,3094,3094,3085,3,6,0,9


In [ ]:
# can also check stats of the bam file with:
# samtools flagstat <file.bam>